In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp
from scipy.constants import k, h, epsilon_0, e, m_e
import os

# Global plotting style
plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'figure.figsize': (10, 6)
})

# Output folder
os.makedirs('plots', exist_ok=True)

# Physical constants (SI)
eps_r = 11.7            # relative permittivity of Si
q = e                   # elementary charge (C)
m_star = 1.08 * m_e     # effective electron mass in Si
E_g = 1.12 * e          # bandgap (J)

# Helper physics functions
def n_i(T):
    """Intrinsic carrier concentration (approximate), SI units (m^-3)."""
    return 2 * (2 * np.pi * m_star * k * T / h**2)**1.5 * np.exp(-E_g / (2 * k * T))

def V_bi_temp(T, N_a, N_d):
    """Built-in potential of a p-n junction (SI)."""
    return (k * T / q) * np.log(N_a * N_d / n_i(T)**2)

def depletion_width_total(Na, Nd, V_bi):
    """
    Total depletion width W = Wp + Wn for an abrupt pn-junction (SI).
    W = sqrt( (2*eps*V_bi/q) * (1/Na + 1/Nd) )
    """
    eps = eps_r * epsilon_0
    return np.sqrt((2 * eps * V_bi / q) * (1.0/Na + 1.0/Nd))

def clustered_mesh(xmin, xmax, n=1600, alpha=6.0):
    """
    Create a mesh clustered around x=0 using a tanh mapping.
    """
    L = max(abs(xmin), abs(xmax))
    s = np.linspace(-1, 1, n)
    x = L * np.tanh(alpha * s) / np.tanh(alpha)
    return x

# Robust Poisson solver
def poisson_solver_auto(N_a, N_d, V_bi, W_pad_factor=6.0, frac_delta=0.05,
                        delta_min=2e-8, delta_max=5e-7, n_mesh=1800,
                        alpha_cluster=6.0, tol=5e-4, max_nodes=200000):
    """
    Auto-selects spatial domain based on depletion width W.
    Returns E(x), phi(x), x.
    """
    eps = eps_r * epsilon_0
    W = depletion_width_total(N_a, N_d, V_bi)
    L = W_pad_factor * (W / 2.0)
    xmin, xmax = -L, L
    x = clustered_mesh(xmin, xmax, n=n_mesh, alpha=alpha_cluster)
    delta = np.clip(frac_delta * W, delta_min, delta_max)

    def equations(x, y):
        phi, E = y
        Nd_profile = N_d * 0.5 * (1 + np.tanh(x / delta))
        Na_profile = N_a * 0.5 * (1 - np.tanh(x / delta))
        rho = q * (Nd_profile - Na_profile)
        return np.vstack((E, rho / eps))

    def bc(ya, yb):
        return np.array([ya[0], yb[0] - V_bi])

    s0 = max(2.0 * delta, 1e-8)
    phi0 = 0.5 * V_bi * (1.0 + np.tanh(x / s0))
    E0 = 0.5 * V_bi * (1.0 / s0) * (1.0 - np.tanh(x / s0)**2)
    y_init = np.vstack((phi0, E0))

    sol = solve_bvp(equations, bc, x, y_init, tol=tol, max_nodes=max_nodes)
    if not sol.success:
        raise RuntimeError(f"Failed to converge: {sol.message}")

    phi = sol.sol(x)[0]
    E = sol.sol(x)[1]
    return E, phi, x

# Figure 1: Temperature dependencies
T_range = np.linspace(200, 400, 120)
Na_T = Nd_T = 1e18 * 1e6  # Convert cm^-3 to m^-3

# (a) V_bi vs T
plt.figure(figsize=(8, 6))
Vbi_vals = [V_bi_temp(T, Na_T, Nd_T) for T in T_range]
plt.plot(T_range, Vbi_vals, linewidth=2)
plt.xlabel('Temperature (K)')
plt.ylabel('Built-in Potential, $V_{bi}$ (V)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('plots/vbi_temperature.png', dpi=300, bbox_inches='tight')
plt.close()

# (b) n_i vs T
plt.figure(figsize=(8, 6))
ni_vals_cm3 = [n_i(T) / 1e6 for T in T_range]  # Convert to cm^-3
plt.plot(T_range, ni_vals_cm3, linewidth=2)
plt.xlabel('Temperature (K)')
plt.ylabel('Intrinsic Carrier Concentration, $n_i$ (cm$^{-3}$)')
plt.yscale('log')
plt.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.savefig('plots/ni_temperature.png', dpi=300, bbox_inches='tight')
plt.close()

# Figure 2: Depletion Width vs Doping
N_range_m3 = np.logspace(22, 26, 220)   # 1e16 to 1e20 cm^-3 in m^-3
V_bi_const = 0.791
W_vals = depletion_width_total(N_range_m3, N_range_m3, V_bi_const)

plt.figure(figsize=(8, 6))
plt.plot(N_range_m3 / 1e6, W_vals, linewidth=2)
plt.xscale('log')
plt.xlabel('Doping Concentration, $N_a = N_d$ (cm$^{-3}$)')
plt.ylabel('Total Depletion Width, $W$ (m)')
plt.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.savefig('plots/depletion_width.png', dpi=300, bbox_inches='tight')
plt.close()

# Figures 3-4: Field and Potential for Symmetric and Asymmetric Junctions
V_bi = 0.791
concentrations_cm3 = [1e16, 1e18, 1e20]
colors = ['tab:blue', 'tab:green', 'tab:red']

# Symmetric junctions - Electric field
plt.figure(figsize=(8, 6))
for i, conc in enumerate(concentrations_cm3):
    N = conc * 1e6
    E, phi, xmesh = poisson_solver_auto(N, N, V_bi)
    plt.plot(xmesh * 1e6, E / 1e3, color=colors[i],
             label=rf'$N_a = N_d = 10^{{{int(np.log10(conc))}}}\ \mathrm{{cm^{{-3}}}}$')
plt.xlabel('Position, $x$ ($\\mu$m)')
plt.ylabel('Electric Field, $E$ (kV/m)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('plots/E_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

# Symmetric junctions - Potential
plt.figure(figsize=(8, 6))
for i, conc in enumerate(concentrations_cm3):
    N = conc * 1e6
    E, phi, xmesh = poisson_solver_auto(N, N, V_bi)
    plt.plot(xmesh * 1e6, phi, color=colors[i],
             label=rf'$N_a = N_d = 10^{{{int(np.log10(conc))}}}\ \mathrm{{cm^{{-3}}}}$')
plt.xlabel('Position, $x$ ($\\mu$m)')
plt.ylabel('Potential, $\\phi$ (V)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('plots/phi_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

# Asymmetric junction
Na_asym = 1e18 * 1e6
Nd_asym = 1e16 * 1e6
E_asym, phi_asym, x_asym = poisson_solver_auto(Na_asym, Nd_asym, V_bi)

# Electric field (asymmetric)
plt.figure(figsize=(8, 6))
plt.plot(x_asym * 1e6, E_asym / 1e3, linewidth=2)
plt.xlabel('Position, $x$ ($\\mu$m)')
plt.ylabel('Electric Field, $E$ (kV/m)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('plots/E_asymmetric.png', dpi=300, bbox_inches='tight')
plt.close()

# Potential (asymmetric)
plt.figure(figsize=(8, 6))
plt.plot(x_asym * 1e6, phi_asym, linewidth=2)
plt.xlabel('Position, $x$ ($\\mu$m)')
plt.ylabel('Potential, $\\phi$ (V)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('plots/phi_asymmetric.png', dpi=300, bbox_inches='tight')
plt.close()

# Figure 5: Sensitivity to smoothing length delta
fracs = np.logspace(-2, -0.3, 12)
rms_values = []
Na_sens = 1e18 * 1e6
Nd_sens = 1e16 * 1e6

E_ref, _, x_ref = poisson_solver_auto(Na_sens, Nd_sens, V_bi, frac_delta=0.5)

for f in fracs:
    E_num, _, x_num = poisson_solver_auto(Na_sens, Nd_sens, V_bi, frac_delta=float(f))
    E_interp = np.interp(x_ref, x_num, E_num)
    rms = np.sqrt(np.mean((E_interp - E_ref)**2))
    rms_values.append(rms)

plt.figure(figsize=(8, 6))
plt.semilogx(fracs, rms_values, 'o-', linewidth=2)
plt.xlabel(r'Smoothing fraction, $f$ in $\delta = f \cdot W$')
plt.ylabel(r'RMS Error vs Reference, $\epsilon_{\mathrm{RMS}}$ (V/m)')
plt.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.savefig('plots/sensitivity_delta.png', dpi=300, bbox_inches='tight')
plt.close()

print("All figures have been generated successfully!")

All figures have been generated successfully!
